### Степаненко Дмитрий Владимирович, тестовое задание в Case Lab ML (АО "Гринатом")

В рамках тестового задания предлагается разработать веб-сервис для оценки комментариев (отзывов) к фильмам.

Рассмотрим датасет IMDB из Stanford
https://ai.stanford.edu/~amaas/data/sentiment/

Более подробное описание структуры файлов данных, а также сами данные можно найти по ссылке: https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

Методология описана в статье:
https://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

### 0. Introduction.

##### C веб-сервисами, в целом, и с реализацией на базе фреймворка Django, в частности, знаком поверхностно в обзорном формате, поэтому попробуем разработать в Jupyter Notebook "сервис для ввода отзыва о фильме с автоматическим присвоением рейтинга (от 1 до 10) и статуса комментария (положительный или отрицательный)".

Цель — создать модель машинного обучения, которая будет предсказывать рейтинг и классифицировать отзыв как положительный или отрицательный.

##### Шаги:

1. Загрузка и подготовка данных:
    - загрузим данные по ссылке, а затем разархивируем их;
    - подготовим текстовые данные для модели (очистка данных, токенизация и приведение к необходимому формату).
2. Обучение модели классификации:
    - построим модель классификации с помощью библиотеки scikit-learn или tensorflow/keras;
    - в качестве метки для классификации используем "положительный/отрицательный" отзыв;
    - для оценки (рейтинг от 1 до 10) можно взять предсказания регрессии на основе тональности текста.
3. Реализация сервиса: мспользуем Jupyter для создания простой формы ввода отзыва, и на выходе будем отображать предсказанную оценку и статус (положительный или отрицательный).

### 1. Подготовка данных.

Мы очистим текст отзывов и создадим метки:
- Положительным отзывам присвоим метку 1.
- Отрицательным отзывам присвоим метку 0.

Затем преобразуем текст в числовой формат с использованием подхода Bag of Words (мешок слов) с помощью CountVectorizer.

In [323]:
# Импортируем необходимые библиотеки
import os
import tarfile
import urllib.request

# Ссылка на набор данных IMDb
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset_folder = "./aclImdb"

# Функция для загрузки и распаковки архива с данными
def download_and_extract_data(url, dest_folder):
    if not os.path.exists(dest_folder):
        # Скачиваем архив
        file_tmp = "./aclImdb_v1.tar.gz"
        urllib.request.urlretrieve(url, file_tmp)
        
        # Распаковываем архив
        with tarfile.open(file_tmp, "r:gz") as tar:
            tar.extractall()
        
        # Удаляем временный файл
        os.remove(file_tmp)
        print(f"Данные успешно загружены и распакованы в папку: {dest_folder}")
    else:
        print(f"Данные уже существуют в папке: {dest_folder}")

# Загружаем и распаковываем данные
download_and_extract_data(url, dataset_folder)


Данные уже существуют в папке: ./aclImdb


In [325]:
# Пути к директориям с положительными и отрицательными отзывами
train_pos_dir = "./aclImdb/train/pos/"
train_neg_dir = "./aclImdb/train/neg/"

# Функция для загрузки отзывов из директории
def load_reviews_from_dir(directory):
    reviews = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), "r", encoding="utf-8") as file:
                reviews.append(file.read())
    return reviews

# Загрузка положительных и отрицательных отзывов
pos_reviews = load_reviews_from_dir(train_pos_dir)
neg_reviews = load_reviews_from_dir(train_neg_dir)

# Вывод количества загруженных отзывов
len(pos_reviews), len(neg_reviews)


(12500, 12500)

### 2. Обучение модели.

Для классификации отзывов используем Naive Bayes, который часто хорошо работает на задачах классификации текста. Мы также проверим качество модели на тестовой выборке.

Шаги для выполнения:
- Преобразование текста в числовые признаки с помощью CountVectorizer.
- Обучение модели Naive Bayes с помощью библиотеки scikit-learn.
- Оценка модели на тестовой выборке.

In [328]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Метки для положительных и отрицательных отзывов
labels_pos = [1] * len(pos_reviews)
labels_neg = [0] * len(neg_reviews)

# Объединяем положительные и отрицательные отзывы и метки
all_reviews = pos_reviews + neg_reviews
all_labels = labels_pos + labels_neg

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(all_reviews, all_labels, test_size=0.2, random_state=42)

# Преобразование текстовых данных в числовые признаки с помощью Bag of Words (CountVectorizer)
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение модели Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)

# Предсказания на тестовой выборке
y_pred = nb_model.predict(X_test_vec)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy * 100:.2f}%")


Точность модели: 84.76%


#### Точность модели в 84.76% означает, что модель правильно классифицирует отзывы (положительные или отрицательные) в 84.76% случаев на тестовой выборке. Это хороший показатель, особенно для задач обработки естественного языка, таких как классификация текстов.

#### Для модели Naive Bayes с простым подходом Bag of Words такой уровень точности считается хорошим, так как это базовый алгоритм для текстовой классификации.

### 3. Проверим модель на аналогичных отзывах на русском языке.

Мы воспользуемся моделью, уже обученной на рецензиях IMDB. Для этого протестируем модель на датасете из 3000 записей, собранных с Кинопоиска. Выборка сбалансирована: содержится примерно по одной тысяче положительных, негативных и нейтральных отзывов (в датасете IMDB присутствуют только положительные и негативные отзывы).

Для удобства работы возьмем готовый набор текстовых файлов, собранный в единый csv-файл (https://github.com/matyushkin/lessons/blob/master/nlp/nlp_datasets/kinopoisk.zip).

##### 1. Подготовка данных:

In [335]:
import zipfile
import os

# Распаковка загруженного архива
zip_file_path = 'kinopoisk.zip'
extract_path = 'kinopoisk/'

# Распаковка архива
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Проверим содержимое папки после распаковки
os.listdir(extract_path)

['kt.csv']

In [336]:
import pandas as pd

# Загрузка файла
csv_file_path = os.path.join(extract_path, 'kt.csv')
data = pd.read_csv(csv_file_path)

# Просмотр первых нескольких строк данных
data.head()


,Unnamed: 0,review,translation,type
0,0,Фильм сняли просто чудно. Disney на этот раз н...,The film was shot just wonderfully. Disney thi...,good
1,1,"На самом деле, к фильму у меня было предубежде...","In fact, I had a bias towards the film. I thou...",good
2,2,"Пожалуй нет ни одного увлеченного киномана, кт...",Perhaps there is not a single keen film fan wh...,good
3,3,Я обожаю старый мультфильм 'Красавица и чудови...,I love the old Beauty and the Beast cartoon. L...,good
4,4,"Никогда не фанател от «Форсажа», для меня он в...",I have never been a fan of Fast and the Furiou...,good


##### 2. Обработка данных и создание модели (токенизация и разделение данных на обучающую и тестовую выборки):

In [338]:
# Преобразуем метки
label_mapping = {'good': 1, 'neutral': 0, 'bad': -1}
data['label'] = data['type'].map(label_mapping)

# Убираем ненужные столбцы для дальнейшего использования
data_cleaned = data[['review', 'label']]

# Проверим полученные данные
data_cleaned.head()

,review,label
0,Фильм сняли просто чудно. Disney на этот раз н...,1
1,"На самом деле, к фильму у меня было предубежде...",1
2,"Пожалуй нет ни одного увлеченного киномана, кт...",1
3,Я обожаю старый мультфильм 'Красавица и чудови...,1
4,"Никогда не фанател от «Форсажа», для меня он в...",1


##### 3. Оценка модели:

In [340]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data_cleaned['review'], data_cleaned['label'], test_size=0.2, random_state=42)

# Преобразование текстовых данных в числовые признаки с помощью Bag of Words (CountVectorizer)
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение модели Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)

# Предсказания на тестовой выборке
y_pred = nb_model.predict(X_test_vec)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 66.17%


#### Точность модели в 66.17% означает, что модель правильно классифицировала отзывы (положительные, нейтральные или негативные) в 66.17% случаев на тестовой выборке. Это ниже, чем 84.76% для набора данных aclImdb, и может указывать на несколько факторов, влияющих на производительность модели:
- Точность — это доля правильных предсказаний из общего числа предсказаний. В данном случае это означает, что модель ошибается в более чем трети случаев.
- Сложность задачи — классификация на три класса (положительные, нейтральные и негативные отзывы) может быть сложнее, чем бинарная классификация, что может объяснять более низкую точность.
- Оценка других метрик — стоит также посмотреть на полноту (recall), точность (precision) и F-мера для более полной картины. Это особенно важно, если классы имеют разные уровни важности.

#### Таким образом, точность 66.17% указывает на необходимость доработки модели и ее компонентов для достижения более надежных результатов.